<a href="https://colab.research.google.com/github/yingzibu/drug_design_JAK/blob/main/sun_lab_compound_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/sun_lab_compound_pk/

In [4]:
# cd /content/drive/MyDrive/A_JAK_design/sun_lab_compound_pk/

/content/drive/MyDrive/A_JAK_design/sun_lab_compound_pk


In [5]:
!pip install rdkit --quiet
!pip install git+https://github.com/samoturk/mol2vec --quiet
from mol2vec.helpers import depict_identifier, plot_2D_vectors, IdentifierTable, mol_to_svg
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import warnings
def warn(*args, **kwargs):
    pass
warnings.warn = warn
from cal import *

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 5.0 MB/s eta 0:00:00


## Make sure 2 xlsx files



*   **compound_data.xlsx**, the excel file containing pk data
*   **drug_smile.xlsx**, the excel file containing SMILES, MW, ALERTS, QED etc.




are in this *directory*. Do not interchange or change the name.

In [6]:
get_dir()

'/content/drive/MyDrive/A_JAK_design/sun_lab_compound_pk/'

In [7]:
ls

cal.py  compound_data.xlsx  drug_smile.xlsx  gfg.html  __pycache__/


In [8]:
compounds

,Compound name,Structure picture,Target,MW,Dose route,Dosage (mg/kg),Time (h),Concentration [ng/(mL or g)],Concentration Average,Tissue,...,ROTB,AROM,ALERTS,QED,LogS,pKa,Notes,Unnamed: 24,Unnamed: 25,Unnamed: 26
0,ARD-3474-WX (36),NaN,Androgen receptor,807.41,PO,3,1,28.9,NaN,Plasma,...,6.0,3.0,1.0,0.263721,-10.890,8.806,PEG400:Labrasol (1:1 v/v),NaN,NaN,NaN
1,ARD-3474-WX (36),NaN,Androgen receptor,807.41,PO,3,3,17.9,NaN,Plasma,...,6.0,3.0,1.0,0.263721,-10.890,8.806,PEG400:Labrasol (1:1 v/v),NaN,NaN,NaN
2,ARD-3474-WX (36),NaN,Androgen receptor,807.41,PO,3,6,11,NaN,Plasma,...,6.0,3.0,1.0,0.263721,-10.890,8.806,PEG400:Labrasol (1:1 v/v),NaN,NaN,NaN
3,ARD-3474-WX (36),NaN,Androgen receptor,807.41,PO,3,24,9.63,NaN,Plasma,...,6.0,3.0,1.0,0.263721,-10.890,8.806,PEG400:Labrasol (1:1 v/v),NaN,NaN,NaN
4,ARD-3474-WX (36),NaN,Androgen receptor,807.41,PO,3,1,10.3,NaN,Plasma,...,6.0,3.0,1.0,0.263721,-10.890,8.806,PEG400:Labrasol (1:1 v/v),NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21338,M-1034-XSL,NaN,Menin,696.815,PO,25,24,4.95,83.535,Plasma,...,11.0,2.0,0.0,0.334316,-7.747,"7.785, 8.087",Red numbers indicate that there were larget di...,137.37,-9.983,7.275
21339,M-1034-XSL,NaN,Menin,696.815,PO,25,1,37.78,101.45,Plasma,...,11.0,2.0,0.0,0.334316,-7.747,"7.785, 8.087",Red numbers indicate that there were larget di...,137.37,-9.983,7.275
21340,M-1034-XSL,NaN,Menin,696.815,PO,25,3,14.79,25.245,Plasma,...,11.0,2.0,0.0,0.334316,-7.747,"7.785, 8.087",Red numbers indicate that there were larget di...,137.37,-9.983,7.275
21341,M-1034-XSL,NaN,Menin,696.815,PO,25,6,7.97,19.57,Plasma,...,11.0,2.0,0.0,0.334316,-7.747,"7.785, 8.087",Red numbers indicate that there were larget di...,137.37,-9.983,7.275


####Check duplicates in drug_smile.xlsx

In [10]:
smi_path = 'drug_smile.xlsx'
smiles_df = pd.read_excel(smi_path)
length = len([*set(smiles_df['NAME'])])

try:
    assert length == smiles_df.shape[0]
except:
    print(f'there are multiple lines for drugs in file {smi_path}')
    for s in set(smiles_df['NAME']):
        if smiles_df.loc[smiles_df['NAME'] == s].shape[0] != 1:
            print(f'multi lines:  {s}      check',
                  smiles_df.loc[smiles_df['NAME'] == s].index.tolist())

### Calculate kp for your sheet

In [ ]:
t_dict = {}
from tqdm import tqdm
invalid_kp_names = []
valid_kp_names = []
one_tissue_names = []
only_plasma_names = []
specials = []
for name in tqdm(names):
    # print(name)
    try:
        data = pk(name)
        l = data.get_kp()
        name, df, tissues, _, dose_routes, doses, time_points, species = data.get_info()
        if len(l) == 0:
            # print(name)
            invalid_kp_names.append(name)
            if len(tissues) == 1:
                one_tissue_names.append(name)
                if tissues[0] == 'Plasma':
                  only_plasma_names.append(name)
            else:
                specials.append(name)
            # name, data_df, tissues, targets, dose_routes, doses, time_points, species = data.get_info()
        # for t in time_points:
        #     if t not in t_dict.keys():
        #         t_dict[t] = 1
        #     else: t_dict[t] += 1
        else:
            valid_kp_names.append(name)
    except:
        print(f'cannot handle {name}')
print()

print(f'total {len(names)} compounds, '
      f"can calculate {len(valid_kp_names)} compounds,",
      f"cannot handle {len(invalid_kp_names)} compounds,\n",
      f"* in invalid calculations {len(one_tissue_names)} compounds have one tissue only \n",
      f"* in those one tissue drugs, {len(only_plasma_names)} compounds have plasma only \n",
      f"* still {len(specials)} compounds need to check")



# specials, 5 compounds indeed no kp:
# 'AK-2288', liver == 0
# 'AK-2278', dosage different, cannot calculate kp
# ARD-3007-WX. BLQ
# M-1403-XRQ, BLQ
# SD-3016 BLQ
################################

  5%|▌         | 73/1382 [00:03<01:29, 14.66it/s]

In [ ]:
len(valid_kp_names)

You can randomly change the ind variable between 0 and len(valid_kp_names)

In [ ]:
ind = 23 #@param {type:"integer"}

if ind == None or ind >= len(valid_kp_names):
    print('invalid input, let ind = 0')
    ind = 0
###############################

name = valid_kp_names[ind]
print(name)
pd.DataFrame.from_dict(pk(name).get_kp())

change the variable "name" to check

In [ ]:
name = "PVT-004206" #@param {type:"string"}
# print(name not in valid_kp_names)
if name == None or (name not in valid_kp_names):
    print(f'invalid name, use {valid_kp_names[0]} instead')
    name = valid_kp_names[0]
data = pk(name)
name, df, tissues, target, dose_routes, doses, time_points, species = data.get_info()
all_kps = data.get_kp()
df = pd.DataFrame.from_dict(all_kps)
print('name: ', name)

tumor_only = True #@param {type:"boolean"}
if tumor_only:
    # If you want tumor only:
    print('tumor only')
    print(df[df['tissue'] == 'Tumor'])
else:
    print(df)

Try PCA

In [ ]:
smi_path = 'drug_smile.xlsx'
smiles_df = pd.read_excel(smi_path)
smiles_df

In [ ]:
# d = 0
in_smile_excel = []
not_in_excel = []

for i in tqdm(valid_kp_names):
    if i in smiles_df['NAME'].tolist():
        in_smile_excel.append(i)
        pass
    else:
        not_in_excel.append(i)
        # print(i)
        # d += 1
# print(d)
print()
print(f'total {len(valid_kp_names)} compounds having kp, \n'
      f"*{len(in_smile_excel)} compounds in {smi_path},\n",
      f"*{len(not_in_excel)} compounds are not in {smi_path}")

For compounds in drug_smile.xlsx, check whether have 24h tumor kp

In [ ]:
from statistics import mean
certain_time = "24" #@param ["24", "6"]
certain_time = int(certain_time)
certain_time_kp_names = []
kp_list = []
tissue = "Tumor" #@param ["Tumor", "Spleen", "Liver"]

new_df = pd.DataFrame()
for i in tqdm(in_smile_excel):
    # df2 = df.loc[:, df.columns != "Duration"]
    features = smiles_df[smiles_df['NAME'] == i].iloc[:, 1:]
    # features = features.loc[:, features.columns != 'SMILES']
    if features.shape[0] > 1:  # duplicates in drug_smile file
        # print(features)
        features = features.iloc[0]
        try:
            features = features.values.reshape(1, int(features.size))
            features = pd.DataFrame(features, columns = smiles_df.columns[3:])
            # features['']
            assert features.shape[0] == 1
        except:
            print('now features,size: ', features.size)
            print('now features.shape: ', features.shape)
            pass
        # print(features)
        # print(features.shape[0])
        # break
    data = pk(i)
    name, df, tissues, target, dose_routes, doses, time_points, species = data.get_info()
    all_kps = data.get_kp()
    df = pd.DataFrame.from_dict(all_kps)
    tumor_df = df.loc[(df['tissue'] == tissue) & (df['time'] == certain_time)]
    if tumor_df.shape[0] != 0:
        # print(tumor_df)
        certain_time_kp_names.append(i)
        new_df = new_df.append([features]*1,ignore_index=True)
        kp_list.append(mean(tumor_df['Kp'].tolist()))
        # print('current df shape', tumor_df.shape[0], len(tumor_df['Kp'].tolist()))
        # print('tumor_df.shape', new_df.shape[0], len(kp_list))
        try:
            assert new_df.shape[0] == len(kp_list)
            assert tumor_df.shape[0] == len(tumor_df['Kp'].tolist())
        except:
            print(i)
            # print('current df shape', tumor_df.shape[0], len(tumor_df['Kp'].tolist()))
            break
print()
print(len(certain_time_kp_names),
      f'compounds have {certain_time}h tumor Kp')
new_df['Kp_average'] = pd.DataFrame(kp_list)
new_df
# print('Kp tumor at 24 h: ')
# print(new_df)

In [ ]:
sns.displot(kp_list, binwidth=3)

Kp range too large, next fig show Kp in Kp range


In [ ]:
kp_min = 0
print("max kp and min kp values: ", new_df['Kp_average'].max(),
      new_df['Kp_average'].min())
print('define your range of kp you want to visualize')
kp_min = new_df['Kp_average'].min()
kp_max = new_df['Kp_average'].max()

In [ ]:
# Max and mininum Kps
# print("max kp and min kp values: ", new_df['Kp'].max(), new_df['Kp'].min())
### change Kp_range here###
kp_min = 0 #@param {type:"number"}
kp_max = 100  #@param {type:"number"}
if kp_min == None:
    kp_min = new_df['Kp_average'].min()
if kp_max == None:
    kp_max = new_df['Kp_average'].max()

Kp_range = [kp_min, kp_max]
###########################

test_df = pd.DataFrame()
test_df = new_df.loc[(new_df['Kp_average'] < Kp_range[1]) & (new_df['Kp_average'] > Kp_range[0])]
test_df = test_df.reset_index(drop=True)

# print(test_df)
features = test_df[test_df.columns[2:-1]]

# print('features before PCA')
# print(features.iloc[187], test_df.iloc[187])
# # print(features)


labels = test_df[test_df.columns[-1]]

# print('labels at 187', labels.iloc[187])

pca = PCA(n_components=2)
features = StandardScaler().fit_transform(features)
pca.fit(features)

# Below is the code for transform the data that Nick has mentioned
# see https://medium.com/all-about-ml/understanding-principal-component-analysis-pca-556778324b0e
# features = StandardScaler().fit_transform(features)

columns = ["principle component 1", "principle component 2"]
features = pd.DataFrame(data = pca.transform(features), columns = columns)
features['label'] = labels

# print(features.iloc[187])
sns.set()
ax = sns.scatterplot(x=columns[0], y=columns[1],
                     hue = 'label', palette='RdBu',
                     data=features)

norm = plt.Normalize(labels.min(), labels.max())
scalarmap = plt.cm.ScalarMappable(cmap="RdBu", norm=norm)
scalarmap.set_array([])

ax.get_legend().remove() # Remove the legend and add a colorbar
ax.figure.colorbar(scalarmap)

# set the title for the figure
plt.title(f"Kp in range {Kp_range}")
plt.show()

In [ ]:
assert len(features)  == len(test_df)

new_features = features.copy()
new_features['NAME'] = test_df['NAME']
new_features['SMILES'] = test_df['SMILES']
# FIRST DEBUG
svgs = []
for smi in tqdm(new_features['SMILES'].tolist()):
    try:
        svg = DrawMol(smi, kekulize=True)
        svgs.append(svg)
    except:
        print(smi)

from bokeh.plotting import figure, show, output_notebook, ColumnDataSource
from bokeh.models import HoverTool, LinearColorMapper, ColumnDataSource
from bokeh.models import ColorBar
from bokeh.sampledata.iris import flowers
output_notebook()
labels = []
for i in new_features[['NAME', 'label']].values:
    # print(i[0], i[1])
    new_labels = f'{i[0]}, Kp_average: {str(round(i[1], 3))}'
    labels.append(new_labels)

source = ColumnDataSource(
                  data=dict(x=new_features.iloc[:, 0] ,
                            y=new_features.iloc[:, 1],
                            col=new_features['label'],
                            desc=labels,
                            svgs=[DrawMol(smi, kekulize=True) for smi \
                                    in new_features['SMILES'].values]))

exp_cmap = LinearColorMapper(palette="Viridis256",
                             low = min(new_features['label']),
                             high = max(new_features['label']))

hover = HoverTool(tooltips="""
    <div>
        <div>@svgs{safe}
        </div>
        <div>
            <span style="font-size: 15px; font-weight: bold;">@desc</span>
        </div>
    </div>
    """
)
p = figure(plot_width=700, plot_height=700,
           tools=['reset,box_zoom,wheel_zoom,zoom_in,zoom_out,pan',hover],
           title="Mouse over the dots")

# p.circle("x", "y",size=5, source=source, line_color=None,
#         fill_color={"field":"col", "transform":exp_cmap})

bar = ColorBar(color_mapper=exp_cmap, location=(0,0))
p.add_layout(bar, "right")

p.circle('x', 'y', size=10, source=source, line_color=None, fill_alpha=0.5,
         fill_color={"field":"col", "transform":exp_cmap});
show(p)

In [ ]:
columns = ['MW', 'ALOGP', 'HBA', 'HBD', 'PSA', 'ROTB', 'AROM',
       'ALERTS', 'QED']
for name in columns:
  fig, ax = plt.subplots()
  # new_df = new_df[new_df['Kp']<200]
  ax.scatter(new_df[name], new_df['Kp_average'], vmin=0, vmax=1)
  plt.xlabel(name)
  plt.ylabel('kp')
  plt.title(f'{name} vs Kp')